In [1]:
import torch
import sys

sys.path.append('../')
import lmy

In [3]:
devices, _ = lmy.getGPU(1, contain_cpu=False)


def run(x):
    """简单计算"""
    return [x.mm(x) for _ in range(10)]


print(devices)
x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4001, 4001), device=devices[1])

# 进行数据预热
run(x_gpu1)
run(x_gpu2)
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

[device(type='cuda', index=0), device(type='cuda', index=1)]


In [4]:
#基准测试
with lmy.Timer("GPU1"):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with lmy.Timer("GPU2"):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

GPU1 has run for 0.1741s
GPU2 has run for 0.1914s


In [5]:
# 我们知道PyTorch支持前后端分离,所以我们可以在不同的GPU上进行计算,而且并不需要对代码进行修改
with lmy.Timer("GPU1 & GPU2"):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 & GPU2 has run for 0.2008s


In [6]:
!whoami

lmy


In [8]:
def copy_to_cpu(x,non_blocking=False):
    return [y.to('cpu',non_blocking=non_blocking) for y in x]


In [14]:
with lmy.Timer('GPU运行'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with lmy.Timer('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

GPU运行 has run for 0.1912s
复制到CPU has run for 0.1500s


In [17]:
with lmy.Timer('GPU运行和拷贝同时进行'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y,True)
    torch.cuda.synchronize()

GPU运行和拷贝同时进行 has run for 0.2067s
